# **Group Assignment** - Bike Sharing

- `instant`: record index
- `dteday` : date
- `season` : season (1:spring, 2:summer, 3:fall, 4:winter)
- `yr` : year (0: 2011, 1:2012)
- `mnth` : month ( 1 to 12)
- `hr` : hour (0 to 23)
- `holiday` : wether day is holiday or not (extracted from http://dchr.dc.gov/page/holiday-schedule)
- `weekday` : day of the week
- `workingday` : if day is neither weekend nor holiday is 1, otherwise is 0.
+ `weathersit` : 
	- 1: Clear, Few clouds, Partly cloudy, Partly cloudy
	- 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
	- 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
	- 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
- `temp` : Normalized temperature in Celsius. The values are divided to 41 (max)
- `atemp`: Normalized feeling temperature in Celsius. The values are divided to 50 (max)
- `hum`: Normalized humidity. The values are divided to 100 (max)
- `windspeed`: Normalized wind speed. The values are divided to 67 (max)
- `casual`: count of casual users
- `registered`: count of registered users
- `cnt`: count of total rental bikes including both casual and registered

In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import itertools

In [2]:
data = pd.read_csv('hour.csv')
data.tail()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
17374,17375,2012-12-31,1,1,12,19,0,1,1,2,0.26,0.2576,0.60,0.1642,11,108,119
17375,17376,2012-12-31,1,1,12,20,0,1,1,2,0.26,0.2576,0.60,0.1642,8,81,89
17376,17377,2012-12-31,1,1,12,21,0,1,1,1,0.26,0.2576,0.60,0.1642,7,83,90
17377,17378,2012-12-31,1,1,12,22,0,1,1,1,0.26,0.2727,0.56,0.1343,13,48,61
17378,17379,2012-12-31,1,1,12,23,0,1,1,1,0.26,0.2727,0.65,0.1343,12,37,49


## PART I: Exploratory Data Analysis

### 1.1 Data Cleaning

#### 1.1.1 Verifying and setting Datatypes
As can be seen from the below table, every value in the table contains the datatype "int64" (integer) or "float64" (float). The only exception is the dteday column which holds the date.

In [3]:
#Checking Data Structure
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     17379 non-null  int64  
 1   dteday      17379 non-null  object 
 2   season      17379 non-null  int64  
 3   yr          17379 non-null  int64  
 4   mnth        17379 non-null  int64  
 5   hr          17379 non-null  int64  
 6   holiday     17379 non-null  int64  
 7   weekday     17379 non-null  int64  
 8   workingday  17379 non-null  int64  
 9   weathersit  17379 non-null  int64  
 10  temp        17379 non-null  float64
 11  atemp       17379 non-null  float64
 12  hum         17379 non-null  float64
 13  windspeed   17379 non-null  float64
 14  casual      17379 non-null  int64  
 15  registered  17379 non-null  int64  
 16  cnt         17379 non-null  int64  
dtypes: float64(4), int64(12), object(1)
memory usage: 2.3+ MB


In [4]:
#Setting instant column as index
data = data.set_index('instant')
#Applying correct Data Types
data['dteday'] = pd.to_datetime(data['dteday'], format='%Y-%m-%d')

#### 1.1.2 Checking for Duplicates, Null and Zero Values
It is essential to verify the completeness and integrity of data. Here we checked for null values and duplicated data. The function **return_rows_with_all_zero_values** checks whether the data contains rows that have all zero values. This is important to check as these rows will not be identified as null in the above column, however they would still need to be cleaned.
- The below analysis reveals that there are no duplicates or null values in the dataset. There are also no columns with all zero values. This means no data needs to be imputed or dropped to ensure data integrity.

In [5]:
#count empty values in columns. This shows there are no rows with empty columns
data.isna().sum()

dteday        0
season        0
yr            0
mnth          0
hr            0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

In [6]:
#Shows how many duplicates there are in the dataframe
data.duplicated().sum()

0

In [7]:
def return_rows_with_all_zero(df, relevant_columns):
    mask = (df[relevant_columns].isna() | (df[relevant_columns] == 0)).all(axis=1) 
    return df[mask]
print(f"There are {len(return_rows_with_all_zero(data, data.select_dtypes(['number']).columns))} rows with all zeros")

There are 0 rows with all zeros


#### 1.1.3 Denormalizing Values
The values in the dataset have been normalized by dividing them by a specific number. As we do not know the context or reason behind this, we are denormalizing them and returning them to their original values in the below code. This also allows a more meaningful exploratory data analysis (EDA).

In [8]:
#temp, atemp, hum, windspeed

data['temp'] = data['temp'] * 41
data['atemp'] = data['atemp'] * 50
data['hum'] = data['hum'] * 100
data['windspeed'] = data['windspeed'] * 67

In [9]:
for col in data.columns:
    print(f"The highest values in column '{col}' is {data[col].max()}")

The highest values in column 'dteday' is 2012-12-31 00:00:00
The highest values in column 'season' is 4
The highest values in column 'yr' is 1
The highest values in column 'mnth' is 12
The highest values in column 'hr' is 23
The highest values in column 'holiday' is 1
The highest values in column 'weekday' is 6
The highest values in column 'workingday' is 1
The highest values in column 'weathersit' is 4
The highest values in column 'temp' is 41.0
The highest values in column 'atemp' is 50.0
The highest values in column 'hum' is 100.0
The highest values in column 'windspeed' is 56.996900000000004
The highest values in column 'casual' is 367
The highest values in column 'registered' is 886
The highest values in column 'cnt' is 977


#### 1.1.4 Verifying Season Behaviour
The dataset claims to split the data by season(1: Spring, 2: Summer, 3: Autumn, 4: Winter). To verify how this is being split we are running the following code. From the output we can see the data is being split by quarter and that at least some labels do not match the description from the dataset. Instead the following seasons apply, based on the following [data](https://www.timeanddate.com/calendar/seasons.html?n=263). The actual seasons is in brackets below:
- The following start and end dates seem to be observable:
    - Season 1 (Winter): 20 December - 20 March
    - Season 2 (Spring): 21 March - 20 June
    - Season 3 (Summer): 21 June - 22 September
    - Season 4 (Autumn) 23 September - 19 December
- We also checked if any data is missing in the time series. None is missing.


In [10]:
for season in data['season'].unique():
    min_day = data[(data['season'] == season) & (data['yr'] == 0)]['dteday'].min()
    max_day = data[(data['season'] == season) & (data['yr'] == 0)]['dteday'].max()
    print(f"The min day for season {season} is {min_day} and the max day is {max_day}")

The min day for season 1 is 2011-01-01 00:00:00 and the max day is 2011-12-31 00:00:00
The min day for season 2 is 2011-03-21 00:00:00 and the max day is 2011-06-20 00:00:00
The min day for season 3 is 2011-06-21 00:00:00 and the max day is 2011-09-22 00:00:00
The min day for season 4 is 2011-09-23 00:00:00 and the max day is 2011-12-20 00:00:00


#### 1.1.5 Checking for missing data in timeseries
To verify the completeness of the time series, data for all days shows be available. The below code ensures there is no missing points in the time series.

In [11]:
# Define the complete date range from the min to max date in the date column
full_date_range = pd.date_range(start=data['dteday'].min(), end=data['dteday'].max(), freq='D')

# Identify the missing dates
missing_dates = full_date_range.difference(data['dteday'])

print("Missing Dates:")
print(missing_dates)

Missing Dates:
DatetimeIndex([], dtype='datetime64[ns]', freq='D')


In [ ]:
import itertools

full_date_range = pd.date_range(start=data['dteday'].min(), end=data['dteday'].max(), freq='D')

# Define the complete range of hours (0-23)
full_hour_range = range(24)

# Create all possible combinations of dates and hours using itertools
all_date_hour_combinations = pd.DataFrame(
    list(itertools.product(full_date_range, full_hour_range)),
    columns=['dteday', 'hr']
)

# Merge with the original data to find missing date-hour combinations
merged = all_date_hour_combinations.merge(data[['dteday', 'hr']], on=['dteday', 'hr'], how='left', indicator=True)

# Filter out the rows that are missing in the original data
missing_date_hours = merged[merged['_merge'] == 'left_only'][['dteday', 'hr']]

print("Missing Date-Hour Combinations:")
missing_date_hours

Missing Date-Hour Combinations:


Index([   29,    50,    51,    75,    99,   123,   147,   243,   244,   267,
       ...
       16039, 16040, 16041, 16042, 16043, 16044, 16251, 16755, 17356, 17379],
      dtype='int64', length=165)

### 1.2 Feature engineering

#### 1.2.1 Adding day variable from date column

In [13]:
#adding day column
data['day_of_month'] = data['dteday'].dt.day
data.head()

,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,day_of_month
instant,,,,,,,,,,,,,,,,,
1,2011-01-01,1,0,1,0,0,6,0,1,9.84,14.395,81.0,0.0,3,13,16,1
2,2011-01-01,1,0,1,1,0,6,0,1,9.02,13.635,80.0,0.0,8,32,40,1
3,2011-01-01,1,0,1,2,0,6,0,1,9.02,13.635,80.0,0.0,5,27,32,1
4,2011-01-01,1,0,1,3,0,6,0,1,9.84,14.395,75.0,0.0,3,10,13,1
5,2011-01-01,1,0,1,4,0,6,0,1,9.84,14.395,75.0,0.0,0,1,1,1


#### 1.2.1 Adding New Seasons
As identifified in the data cleaning process, the season column is encoded in a strange way. As such new columns are added to better consider the seasons. Which column is the best to use can be later determined in the model training and evaluation process.

In [14]:
data['fiscal_quarter'] = data['dteday'].dt.quarter


#### 1.2.2 Rush Hour Variable.
Data retreived from: https://www.commuterconnections.org/wp-content/uploads/SOC-2022-At-a-Glance-Commute-Patterns.pdf

where:
- 0 = Early Morning (12 AM - 6 AM)
- 1 = Morning Rush (6 AM - 10 AM)
- 2 = Midday (10 AM - 3 PM)
- 3 = Evening Rush (3 PM - 7 PM)
- 4 = Night (7 PM - 12 AM)

In [15]:
def categorize_hour(hour):
    if 0 <= hour < 6:
        return 0
    elif 6 <= hour < 10:
        return 1
    elif 10 <= hour < 15:
        return 2
    elif 15 <= hour < 19:
        return 3
    else:
        return 4

data['hour_category'] = data['hr'].apply(categorize_hour)

#### 1.2.3 Rain/Snow Variable
Where: 0 = no rain or snow | 1 = rain or snow

In [ ]:
def categorize_weather(weathersit):
    if 1 <= weathersit < 3:
        return 0
    else:
        return 1

data['rain_snow'] = data['weathersit'].apply(categorize_weather)


### 1.3 Exploratory Data Analysis (EDA)

## PART II: Prediction Model

## PART III: Streamlit dashboard

In [16]:
### This part goes in a separate script ###